In [1]:
import concurrent.futures
import os
import requests
import json

In [2]:
def get_ely_code(road_number, road_section, distance_from_road_section_start):
    try:
        url = f"https://julkinen.vayla.fi/viitekehysmuunnin/muunna?tie={road_number}&osa={road_section}&etaisyys={distance_from_road_section_start}&valihaku=false&palautusarvot=4"
        response = requests.get(url, timeout=10)
        return response.json()["features"][0]["properties"]["ely"]
    except Exception as e:
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(e).__name__, e.args)
        print (message, road_number, road_section, distance_from_road_section_start)

In [3]:
json_location = "../visualisointi/tms-stations.json"
with open(json_location, 'r') as myfile:
    data = myfile.read()

tms_data = json.loads(data)["features"]

In [4]:
def get_ely_for_tms(tms):
    road_data = tms["properties"]["roadAddress"]
    road_nr = road_data["roadNumber"]
    road_section = road_data["roadSection"]
    distance = road_data["distanceFromRoadSectionStart"]
    ely_code = get_ely_code(road_nr, road_section, distance)
    return ely_code

In [5]:
def addOrAppendAtKey(key, value_to_add, dictionary):
    value = dictionary.get(key, None)
    if value is not None: 
        value.append(value_to_add)
    else:
        dictionary[key] = [value_to_add]

In [6]:
mapping = {}

In [7]:
for tms in tms_data:
    ely = get_ely_for_tms(tms)
    tms_id = tms["properties"]["tmsNumber"]
    if ely is not None:
        addOrAppendAtKey(ely, tms_id, mapping)

An exception of type KeyError occurred. Arguments:
('ely',) 51 2 3797
An exception of type KeyError occurred. Arguments:
('ely',) 5 133 6185
An exception of type KeyError occurred. Arguments:
('ely',) 4 303 3210


In [70]:
with open('ely_lamid_mapping.json', 'w') as fp:
    json.dump(mapping, fp, sort_keys=True)


In [9]:
len(tms_data)

497

In [14]:
length = 0
for key in mapping.keys():
    length += len(mapping[key])

In [15]:
length

494